In [50]:
#pip install tensorflow_hub
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [51]:
df=pd.read_csv('reviews.csv')

In [52]:
df.head()

,Unnamed: 0,review_id,app_id,review_text,label
0,0,136524320,2008820,i can confirm that the game contains a knight ...,1
1,1,136524365,2008820,"Really good game, enjoyed every second of it",1
2,2,116734222,1729900,Its not finished yet but honestly it has great...,1
3,3,116989907,1729900,"Hey. It's really good. Songs are good, Gamepla...",1
4,4,136568878,307950,Fun so far,1


In [54]:
df.groupby('label').count() # there is class imbalance

,Unnamed: 0,review_id,app_id,review_text
label,,,,
0,120,120,120,120
1,606,606,606,606


In [8]:
# Load pre-trained language model
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

2023-04-15 14:19:59.552869: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [55]:
X_train, X_test, y_train, y_test = train_test_split(df["review_text"], df["label"], test_size=0.2, 
                                                    # stratify will make sure the labels are
                                                    # well proportioned in train and test
                                                    stratify=df["label"], random_state=42)


In [56]:
sum(y_train)/len(y_train) # check imbalance

0.8344827586206897

In [57]:
sum(y_test)/len(y_test) # check imbalance

0.8356164383561644

In [58]:
# Prepare data
X_train = [text.lower() for text in X_train] # convert to lowercase
X_test = [text.lower() for text in X_test] # convert to lowercase
X_train_embeddings = embed(X_train) # generate embeddings for train data
X_test_embeddings = embed(X_test) # generate embeddings for test data

In [59]:
X_train_embeddings.shape

TensorShape([580, 512])

In [60]:
X_test_embeddings.shape

TensorShape([146, 512])

In [64]:
# Train logistic regression model
# change betweeen models to see difference
clf = LogisticRegression().fit(X_train_embeddings, y_train)
clf = SVC(kernel='linear', C=1, random_state=42).fit(X_train_embeddings, y_train)

# Make predictions on test set
y_pred = clf.predict(X_test_embeddings)

# Evaluate model
accuracy = clf.score(X_test_embeddings, y_test)

In [65]:
accuracy

0.8767123287671232

In [66]:
# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[  8,  16],
       [  2, 120]])